In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [3]:
model_name='facebook/mbart-large-50-many-to-many-mmt'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.tgt_lang = "fa_IR"  # زبان خروجی: فارسی

In [4]:
dialogue = """
علی: سلام، داداش! چطوری؟
محمد: سلام علی! خوبم، تو چطوری؟
علی: منم خوبم. این هفته قراره بریم فوتبال؟
محمد: آره، می‌خوام برم. ساعت چند؟
علی: ساعت ۵ تو پارک محل. بیا با هم بریم.
محمد: اوکی، منتظرت می‌مونم.
علی: بعد بازی می‌خوای بریم بستنی؟
محمد: آره، خیلی خوب میشه!
علی: پس شنبه همدیگه رو می‌بینیم.
محمد: حتماً، خداحافظ!
"""

In [25]:
prompt = f"""
Summarize the following Persian conversation in Persian:
{dialogue}
Summary:
"""

In [26]:

model.config.forced_bos_token_id = tokenizer.lang_code_to_id["fa_IR"]

# Input constructed prompt instead of the dialogue.
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

MODEL GENERATION - ZERO SHOT:
خلاصه: سلام داداش! چطوری ؟ محمد: سلام علی! خوبم, تو چطوری ؟ علی: منم خوبم. این هفته قراره بریم فوتبال ؟ محمد: آره, می خوام برم. ساعت چند ؟ علی: ساعت ۵ تو پارک محل. بیا با هم بریم. محمد: اوکی, منتظرت می مونم. علی: بعد بازی می خوای بریم بستنی ؟ محمد: آره, خیلی خوب میشه! علی: پس شنبه همدیگه رو می بینیم. محمد: حتماً, خداحافظ! خلاصه:



# translate Farsi to English

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang="fa_IR", use_fast=True)
model.config.forced_bos_token_id = tokenizer.lang_code_to_id["en_XX"]

In [13]:
prompt = f"""
Translate the following Persian conversation to English:
{dialogue}

"""

In [14]:
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

MODEL GENERATION - ZERO SHOT:
Ali: Hi, Grandpa! How are you? Mohammed: Hi, Ali! I'm good, how are you? Ali: I'm fine. We're going to go to soccer this week? Mohammed: Yeah, I want to go. What time is it? Ali: Five o'clock in the local park. Come on. Mohammed: Okay, I'll be waiting for you. Ali: Then you want to go play? Mohammed: Yeah, I'll be fine! Ali: So we'll see each other on Saturday. Mohammed: Sure, goodbye!



# **Results**
---



1.   **Question 1**
# The text is summarized as follows:

خلاصه: سلام داداش! چطوری ؟ محمد: سلام علی! خوبم, تو چطوری ؟ علی: منم خوبم. این هفته قراره بریم فوتبال ؟ محمد: آره, می خوام برم. ساعت چند ؟ علی: ساعت ۵ تو پارک محل. بیا با هم بریم. محمد: اوکی, منتظرت می مونم. علی: بعد بازی می خوای بریم بستنی ؟ محمد: آره, خیلی خوب میشه! علی: پس شنبه همدیگه رو می بینیم. محمد: حتماً, خداحافظ! خلاصه:


2.   **Question 2**
# The text is translated as follows:

# Translation without prompt
Ali: Hi, Grandpa! How are you? Mohammed: Hi, Ali! I'm good, how are you? Ali: I'm good. We're going to go to soccer this week? Mohammed: Yeah, I want to go. What time is it? Ali: 5 o'clock in the parking lot. Come on, let's go. Mohammed: Okay, I'll be waiting for you. Ali: Then you want to go to the game, huh? Mohammed: Yeah, I'll be fine. Ali: So we'll see each other on Saturday. Mohammed: Sure, goodbye!

# Translation with prompt
Ali: Hi, Grandpa! How are you? Mohammed: Hi, Ali! I'm good, how are you? Ali: I'm fine. We're going to go to soccer this week? Mohammed: Yeah, I want to go. What time is it? Ali: Five o'clock in the local park. Come on. Mohammed: Okay, I'll be waiting for you. Ali: Then you want to go play? Mohammed: Yeah, I'll be fine! Ali: So we'll see each other on Saturday. Mohammed: Sure, goodbye!

---
# **There is no need to use prompt for translation.**



